In [ ]:
# Carnival Core Score (CCS) - Macro Scanner
# Quantitative momentum/trend analysis system

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("Carnival Core Score (CCS) - Macro Scanner")
print("="*80)
print(f"Initialized: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [ ]:
# ============================================================================
# TICKER CONFIGURATION - Editable Lists
# ============================================================================

TICKERS = {
    'MACRO': [
        # Equity Index Futures
        'ES=F',      # S&P 500 Futures
        'NQ=F',      # Nasdaq Futures
        'RTY=F',     # Russell 2000 Futures
        
        # Treasury Futures
        'ZT=F',      # 2-Year Treasury
        'ZF=F',      # 5-Year Treasury
        'ZN=F',      # 10-Year Treasury
        'ZB=F',      # 30-Year Treasury
        
        # Currency Pairs
        'DX-Y.NYB',  # US Dollar Index
        'USDJPY=X',  # USD/JPY
        'EURUSD=X',  # EUR/USD
        'GBPUSD=X',  # GBP/USD
        'AUDUSD=X',  # AUD/USD
        'USDCAD=X',  # USD/CAD
        
        # Commodities
        'CL=F',      # Crude Oil
        'NG=F',      # Natural Gas
        'HG=F',      # Copper
        'PA=F',      # Palladium
        'PL=F',      # Platinum
        'GC=F',      # Gold
        'SI=F',      # Silver
        'BTC-USD',   # Bitcoin
    ],
    
    'SECTORS': [
        'XLK',   # Technology
        'XLV',   # Health Care
        'XLF',   # Financials
        'XLY',   # Consumer Discretionary
        'XLC',   # Communication Services
        'XLI',   # Industrials
        'XLP',   # Consumer Staples
        'XLE',   # Energy
        'XLU',   # Utilities
        'XLB',   # Materials
        'XLRE',  # Real Estate
        'XHB',   # Homebuilders
        'XBI',   # Biotech
        'SMH',   # Semiconductors
        'SPHB',  # High Beta
        'SPLV',  # Low Volatility
    ],
    
    'TOP_STOCKS': [
        'AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'TSLA', 'GOOGL',
        'AVGO', 'GOOG', 'BRK-B', 'JPM', 'LLY', 'V', 'XOM',
        'UNH', 'MA', 'COST', 'HD', 'WMT', 'PG', 'NFLX',
        'JNJ', 'BAC', 'ABBV', 'CRM', 'AMD',
    ],
    
    'WORLD_ETFS': [
        'VEA',   # Developed Markets ex-US
        'IEMG',  # Emerging Markets
        'EEM',   # Emerging Markets (Legacy)
        'ACWI',  # Global All-Cap
        'BNDX',  # International IG Bonds (Hedged)
        'VWOB',  # EM USD Sovereign Bonds
    ],
}

# Display names for cleaner output
TICKER_NAMES = {
    'ES=F': 'ES',
    'NQ=F': 'NQ',
    'RTY=F': 'RTY',
    'ZT=F': 'ZT (2y)',
    'ZF=F': 'ZF (5y)',
    'ZN=F': 'ZN (10y)',
    'ZB=F': 'ZB (30y)',
    'DX-Y.NYB': 'DXY',
    'USDJPY=X': 'USDJPY',
    'EURUSD=X': 'EURUSD',
    'GBPUSD=X': 'GBPUSD',
    'AUDUSD=X': 'AUDUSD',
    'USDCAD=X': 'USDCAD',
    'CL=F': 'Oil',
    'NG=F': 'Nat Gas',
    'HG=F': 'Copper',
    'PA=F': 'Palladium(PA)',
    'PL=F': 'Platinum',
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'BTC-USD': 'BTC',
    'XLK': 'XLK (Tech)',
    'XLV': 'XLV (Health)',
    'XLF': 'XLF (Financials)',
    'XLY': 'XLY (Disc)',
    'XLC': 'XLC (Comm)',
    'XLI': 'XLI (Ind)',
    'XLP': 'XLP (Stap)',
    'XLE': 'XLE (Energy)',
    'XLU': 'XLU (Util)',
    'XLB': 'XLB (Mat)',
    'XLRE': 'XLRE (RE)',
    'XHB': 'XHB (Home)',
    'XBI': 'XBI (Bio)',
    'SMH': 'SMH (Semi)',
    'SPHB': 'SPHB (HiB)',
    'SPLV': 'SPLV (LoVol)',
    'VEA': 'VEA (Dev ex-US Eq)',
    'IEMG': 'IEMG (EM Eq)',
    'EEM': 'EEM (EM Eq - Legacy)',
    'ACWI': 'ACWI (Global Eq)',
    'BNDX': 'BNDX (Int\'l IG Bonds Hdg)',
    'VWOB': 'VWOB (EM USD Sov Bonds)',
}

print("Ticker configuration loaded:")
for category, tickers in TICKERS.items():
    print(f"  {category}: {len(tickers)} instruments")
print(f"\nTotal instruments: {sum(len(v) for v in TICKERS.values())}")

In [ ]:
# ============================================================================
# DATA FETCHING MODULE
# ============================================================================

def fetch_market_data(ticker, period='90d'):
    """
    Fetch historical price data for a given ticker.
    
    Args:
        ticker: Yahoo Finance ticker symbol
        period: Historical period to fetch (default 90 days)
    
    Returns:
        DataFrame with OHLCV data
    """
    try:
        data = yf.download(ticker, period=period, progress=False)
        if data.empty or len(data) < 20:
            return None
        
        # Handle multi-index columns (yfinance sometimes returns this)
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)
        
        # Verify we have Close column
        if 'Close' not in data.columns:
            return None
            
        return data
    except Exception as e:
        return None

def fetch_all_data(ticker_dict):
    """
    Fetch data for all tickers in the configuration.
    
    Args:
        ticker_dict: Dictionary of ticker categories
    
    Returns:
        Dictionary of DataFrames keyed by ticker
    """
    all_data = {}
    total_tickers = sum(len(v) for v in ticker_dict.values())
    
    print(f"\nFetching data for {total_tickers} instruments...")
    print("-" * 80)
    
    for category, tickers in ticker_dict.items():
        print(f"\n{category}:")
        for ticker in tickers:
            display_name = TICKER_NAMES.get(ticker, ticker)
            print(f"  Fetching {display_name}...", end=' ')
            
            data = fetch_market_data(ticker)
            if data is not None:
                all_data[ticker] = data
                print(f"✓ ({len(data)} days)")
            else:
                print("✗ (insufficient data)")
    
    print(f"\n{'-' * 80}")
    print(f"Successfully loaded: {len(all_data)}/{total_tickers} instruments")
    
    return all_data

print("Data fetching module loaded")

In [ ]:
# ============================================================================
# TECHNICAL INDICATORS ENGINE
# ============================================================================

def calculate_roc(prices, period):
    """Calculate Rate of Change over specified period"""
    return ((prices - prices.shift(period)) / prices.shift(period)) * 100

def calculate_rsi(prices, period=14):
    """Calculate Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_ma_slope(prices, period):
    """Calculate moving average slope (momentum)"""
    ma = prices.rolling(window=period).mean()
    slope = ((ma - ma.shift(1)) / ma.shift(1)) * 100
    return slope

def calculate_price_vs_ma(prices, ma_period):
    """Calculate price position relative to moving average"""
    ma = prices.rolling(window=ma_period).mean()
    return ((prices - ma) / ma) * 100

def calculate_technical_indicators(df):
    """
    Calculate all technical indicators for scoring including volume analysis.
    
    Args:
        df: DataFrame with OHLC data
    
    Returns:
        DataFrame with added indicator columns
    """
    close = df['Close']
    
    # Rate of Change indicators (multiple timeframes)
    df['ROC_1'] = calculate_roc(close, 1)
    df['ROC_2'] = calculate_roc(close, 2)
    df['ROC_5'] = calculate_roc(close, 5)
    df['ROC_10'] = calculate_roc(close, 10)
    df['ROC_20'] = calculate_roc(close, 20)
    
    # Moving Average indicators
    df['SMA_5'] = close.rolling(window=5).mean()
    df['SMA_10'] = close.rolling(window=10).mean()
    df['SMA_20'] = close.rolling(window=20).mean()
    df['SMA_50'] = close.rolling(window=50).mean()
    
    # MA Slopes (trend direction)
    df['MA5_Slope'] = calculate_ma_slope(close, 5)
    df['MA10_Slope'] = calculate_ma_slope(close, 10)
    df['MA20_Slope'] = calculate_ma_slope(close, 20)
    
    # Price vs MA (position)
    df['Price_vs_MA5'] = calculate_price_vs_ma(close, 5)
    df['Price_vs_MA10'] = calculate_price_vs_ma(close, 10)
    df['Price_vs_MA20'] = calculate_price_vs_ma(close, 20)
    
    # RSI
    df['RSI'] = calculate_rsi(close, 14)
    
    # MA Crossovers (1 = bullish, -1 = bearish, 0 = neutral)
    df['MA_Cross_5_10'] = np.where(df['SMA_5'] > df['SMA_10'], 1, -1)
    df['MA_Cross_10_20'] = np.where(df['SMA_10'] > df['SMA_20'], 1, -1)
    
    # Volume-based indicators (check if volume exists - forex doesn't have volume)
    if 'Volume' in df.columns and df['Volume'].sum() > 0:
        volume = df['Volume']
        df['Vol_SMA_5'] = volume.rolling(window=5).mean()
        df['Vol_SMA_20'] = volume.rolling(window=20).mean()
        
        # Volume ratio (current vs 20-day average)
        df['Vol_Ratio'] = volume / df['Vol_SMA_20']
        
        # Volume trend (5-day vs 20-day MA)
        df['Vol_Trend'] = df['Vol_SMA_5'] / df['Vol_SMA_20']
        
        # Volume-Price correlation (price direction * volume strength)
        df['Vol_Price_Corr'] = np.sign(df['ROC_1']) * (df['Vol_Ratio'] - 1)
    else:
        # For forex or instruments without volume, set neutral values
        df['Vol_Ratio'] = 1.0
        df['Vol_Trend'] = 1.0
        df['Vol_Price_Corr'] = 0.0
    
    return df

print("Technical indicators engine loaded (WITH VOLUME ANALYSIS)")
print("Price Indicators: ROC (1,2,5,10,20d), MA (5,10,20,50), RSI, MA Crossovers")
print("Volume Indicators: Vol Ratio, Vol Trend, Vol-Price Correlation")

In [ ]:
# ============================================================================
# COMPOSITE SCORING ALGORITHM (WITH VOLUME CONVICTION)
# ============================================================================

def normalize_to_range(values, min_val=-10, max_val=10):
    """
    Normalize values with aggressive scaling to match reference output.
    Handles extreme values gracefully.
    """
    # Clip extreme values
    values = np.clip(values, -100, 100)
    
    # Use tanh for smooth normalization with aggressive scaling
    # Changed from /10 to /3 for much stronger signals
    normalized = np.tanh(values / 3) * max_val
    return normalized

def calculate_daily_score(df_row):
    """
    Calculate composite momentum score with volume conviction.
    
    Score components (total 100%):
    - ROC (Price Momentum): 35%
    - MA Trend: 25%
    - Volume & Conviction: 20%
    - RSI: 12%
    - Price vs MA: 8%
    
    Returns:
        Score from -10 (very bearish) to +10 (very bullish)
    """
    
    # Component 1: ROC Signals (35%) - medium-term trends over daily noise
    roc_score = (
        df_row['ROC_1'] * 0.15 +    # Reduced from 0.35 - less weight on daily noise
        df_row['ROC_2'] * 0.15 +    # Reduced from 0.25
        df_row['ROC_5'] * 0.25 +    # Increased from 0.20
        df_row['ROC_10'] * 0.25 +   # Increased from 0.12 - more weight on trends
        df_row['ROC_20'] * 0.20     # Increased from 0.08
    )
    roc_score = normalize_to_range(roc_score) * 0.35
    
    # Component 2: MA Trend Signals (25%)
    ma_slope_score = (
        df_row['MA5_Slope'] * 0.40 +
        df_row['MA10_Slope'] * 0.35 +
        df_row['MA20_Slope'] * 0.25
    )
    ma_cross_score = (df_row['MA_Cross_5_10'] + df_row['MA_Cross_10_20']) * 2.5
    ma_score = (normalize_to_range(ma_slope_score) * 0.7 + ma_cross_score * 0.3) * 0.25
    
    # Component 3: Volume & Conviction (20%)
    # Sub-component 3a: Volume ratio vs average (40% of 20% = 8%)
    vol_ratio_score = normalize_to_range((df_row['Vol_Ratio'] - 1) * 100) * 0.40
    
    # Sub-component 3b: Volume trend (30% of 20% = 6%)
    vol_trend_score = normalize_to_range((df_row['Vol_Trend'] - 1) * 100) * 0.30
    
    # Sub-component 3c: Volume-Price correlation (30% of 20% = 6%)
    vol_price_corr_score = normalize_to_range(df_row['Vol_Price_Corr'] * 100) * 0.30
    
    volume_score = (vol_ratio_score + vol_trend_score + vol_price_corr_score) * 0.20
    
    # Component 4: RSI Momentum (12%)
    # Convert RSI to -10 to +10 scale (50 = neutral)
    rsi_score = ((df_row['RSI'] - 50) / 5)
    rsi_score = np.clip(rsi_score, -10, 10) * 0.12
    
    # Component 5: Price vs MA (8%)
    price_ma_score = (
        df_row['Price_vs_MA5'] * 0.40 +
        df_row['Price_vs_MA10'] * 0.35 +
        df_row['Price_vs_MA20'] * 0.25
    )
    price_ma_score = normalize_to_range(price_ma_score) * 0.08
    
    # Combine all components
    total_score = roc_score + ma_score + volume_score + rsi_score + price_ma_score
    
    # Final clipping
    total_score = np.clip(total_score, -10, 10)
    
    return total_score

def calculate_scores(df):
    """
    Calculate daily scores for entire dataframe.
    
    Args:
        df: DataFrame with technical indicators
    
    Returns:
        DataFrame with added 'Score' column
    """
    df = df.copy()
    
    # Calculate score for each row
    df['Score'] = df.apply(calculate_daily_score, axis=1)
    
    # Apply smoothing using EMA (alpha=0.3 for smoothing)
    df['Score'] = df['Score'].ewm(span=3, adjust=False).mean()
    
    # Round to 1 decimal place
    df['Score'] = df['Score'].round(1)
    
    return df

print("Composite scoring algorithm loaded (CALIBRATED + VOLUME)")
print("")
print("Score Allocation:")
print("  - ROC (Price Momentum):  35%")
print("  - MA Trend:              25%")
print("  - Volume & Conviction:   20%")
print("  - RSI:                   12%")
print("  - Price vs MA:            8%")
print("")
print("ROC Weighting: Reduced daily noise, increased 5-20 day trend weight")
print("Normalization: More aggressive (tanh / 3) for stronger signals")
print("Volume Analysis: Confirms conviction behind price moves")
print("Output range: -10 (Very Bearish) to +10 (Very Bullish)")

In [ ]:
# ============================================================================
# SENTIMENT CLASSIFICATION & ROLLING AVERAGES
# ============================================================================

def classify_sentiment(score):
    """
    Classify score into sentiment category.
    
    Returns:
        Tuple of (sentiment_text, color_code)
    """
    if score >= 7.0:
        return ("Very Bullish (L/S)", "green_bold")
    elif score >= 4.0:
        return ("Bullish (L)", "green")
    elif score > 1.0:
        return ("Bullish (L)", "green")
    elif score >= -1.0:
        return ("Neutral (L/S chop)", "white")
    elif score >= -4.0:
        return ("Neutral (L/S chop)", "white")
    elif score >= -7.0:
        return ("Bearish (S)", "red")
    else:
        return ("Very Bearish (L/S)", "red_bold")

def calculate_rolling_metrics(df):
    """
    Calculate rolling averages and historical scores.
    
    Args:
        df: DataFrame with Score column
    
    Returns:
        Dictionary with current and historical metrics
    """
    if 'Score' not in df.columns or len(df) < 5:
        return None
    
    # Get the most recent scores
    recent_scores = df['Score'].tail(25)  # Last 25 days for calculations
    
    # Current score and historical daily scores
    current_score = recent_scores.iloc[-1] if len(recent_scores) >= 1 else np.nan
    score_d1 = recent_scores.iloc[-2] if len(recent_scores) >= 2 else np.nan
    score_d2 = recent_scores.iloc[-3] if len(recent_scores) >= 3 else np.nan
    score_d3 = recent_scores.iloc[-4] if len(recent_scores) >= 4 else np.nan
    score_d4 = recent_scores.iloc[-5] if len(recent_scores) >= 5 else np.nan
    
    # Rolling averages
    avg_5d = recent_scores.tail(5).mean() if len(recent_scores) >= 5 else np.nan
    avg_10d = recent_scores.tail(10).mean() if len(recent_scores) >= 10 else np.nan
    avg_20d = recent_scores.tail(20).mean() if len(recent_scores) >= 20 else np.nan
    
    # Sentiment classification
    sentiment, color = classify_sentiment(current_score)
    
    return {
        'score': current_score,
        'sentiment': sentiment,
        'color': color,
        'score_d1': score_d1,
        'score_d2': score_d2,
        'score_d3': score_d3,
        'score_d4': score_d4,
        'avg_5d': avg_5d,
        'avg_10d': avg_10d,
        'avg_20d': avg_20d,
    }

def process_all_tickers(market_data):
    """
    Process all tickers: calculate indicators, scores, and metrics.
    
    Args:
        market_data: Dictionary of DataFrames with raw price data
    
    Returns:
        Dictionary with processed results for each ticker
    """
    results = {}
    
    print("\nProcessing indicators and scores...")
    print("-" * 80)
    
    for ticker, df in market_data.items():
        try:
            # Calculate technical indicators
            df_with_indicators = calculate_technical_indicators(df.copy())
            
            # Calculate composite scores
            df_with_scores = calculate_scores(df_with_indicators)
            
            # Calculate rolling metrics
            metrics = calculate_rolling_metrics(df_with_scores)
            
            if metrics is not None:
                results[ticker] = metrics
                
        except Exception as e:
            print(f"  Error processing {ticker}: {str(e)}")
            continue
    
    print(f"Successfully processed: {len(results)} instruments")
    print("-" * 80)
    
    return results

print("Sentiment classification loaded")
print("\nSentiment Levels:")
print("  Very Bullish (L/S): Score >= 7.0")
print("  Bullish (L):        Score 4.0 - 6.9")
print("  Neutral (L/S chop): Score -4.0 - 3.9")
print("  Bearish (S):        Score -7.0 - -4.1")
print("  Very Bearish (L/S): Score <= -7.0")

In [ ]:
# ============================================================================
# TERMINAL OUTPUT FORMATTER
# ============================================================================

class Colors:
    """ANSI color codes for terminal output"""
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    MAGENTA = '\033[95m'
    CYAN = '\033[96m'
    WHITE = '\033[97m'
    BOLD = '\033[1m'
    RESET = '\033[0m'
    
def colorize(text, color_code):
    """Apply color to text based on color code"""
    color_map = {
        'red_bold': f"{Colors.BOLD}{Colors.RED}",
        'red': Colors.RED,
        'green_bold': f"{Colors.BOLD}{Colors.GREEN}",
        'green': Colors.GREEN,
        'white': Colors.WHITE,
    }
    color = color_map.get(color_code, Colors.WHITE)
    return f"{color}{text}{Colors.RESET}"

def format_score(value):
    """Format score value to match reference output"""
    if pd.isna(value):
        return "  "
    # Convert to int if it's a whole number, otherwise 1 decimal
    if abs(value - round(value)) < 0.05:
        return f"{int(round(value))}"
    return f"{value:.1f}"

def print_category_table(category_name, results, ticker_list):
    """
    Print formatted table for a category of tickers.
    
    Args:
        category_name: Name of the category (e.g., "MACRO", "SECTORS")
        results: Dictionary of processed results
        ticker_list: List of tickers in this category
    """
    print(f"\n=== Carnival Core Score (CCS) - {category_name} ===")
    
    # Header line
    header = f"{'Ticker':<20} {'Score':>5} {'Sentiment':<25} {'Avg Score 5D':>12} {'Score D-1':>10} {'Score D-2':>10} {'Score D-3':>10} {'Score D-4':>10} {'Avg Score 10D':>13} {'Avg Score 20D':>13}"
    print(header)
    print("-" * 150)
    
    # Data rows
    for ticker in ticker_list:
        if ticker not in results:
            continue
            
        metrics = results[ticker]
        display_name = TICKER_NAMES.get(ticker, ticker)
        
        # Format score with proper alignment
        score_val = metrics['score']
        if pd.isna(score_val):
            score_str = "    "
        elif abs(score_val - round(score_val)) < 0.05:
            score_str = f"{int(round(score_val)):4d}"
        else:
            score_str = f"{score_val:4.1f}"
        
        sentiment_colored = colorize(metrics['sentiment'], metrics['color'])
        
        # Format all numeric values
        avg_5d_str = format_score(metrics['avg_5d'])
        d1_str = format_score(metrics['score_d1'])
        d2_str = format_score(metrics['score_d2'])
        d3_str = format_score(metrics['score_d3'])
        d4_str = format_score(metrics['score_d4'])
        avg_10d_str = format_score(metrics['avg_10d'])
        avg_20d_str = format_score(metrics['avg_20d'])
        
        row = (
            f"{display_name:<20} "
            f"{score_str:>5} "
            f"{sentiment_colored:<35} "  # Extra space for ANSI codes
            f"{avg_5d_str:>12} "
            f"{d1_str:>10} "
            f"{d2_str:>10} "
            f"{d3_str:>10} "
            f"{d4_str:>10} "
            f"{avg_10d_str:>13} "
            f"{avg_20d_str:>13}"
        )
        
        print(row)

def generate_report(results, ticker_dict):
    """
    Generate complete CCS report for all categories.
    
    Args:
        results: Dictionary of processed results
        ticker_dict: Dictionary of ticker categories
    """
    print("\n" + "=" * 150)
    print("CARNIVAL CORE SCORE (CCS) - DAILY REPORT")
    print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 150)
    
    # Print each category
    category_names = {
        'MACRO': 'MACRO',
        'SECTORS': 'SECTORS',
        'TOP_STOCKS': 'TOP STOCKS',
        'WORLD_ETFS': 'WORLD ETFS (Charts Only)'
    }
    
    for category in ['MACRO', 'SECTORS', 'TOP_STOCKS', 'WORLD_ETFS']:
        if category in ticker_dict:
            print_category_table(category_names[category], results, ticker_dict[category])
    
    print("\n" + "=" * 150)
    print("END OF REPORT")
    print("=" * 150)

print("Terminal output formatter loaded")
print("Color coding: Green (Bullish), Red (Bearish), White (Neutral)")

In [ ]:
# ============================================================================
# MAIN EXECUTION - Run Daily CCS Report
# ============================================================================

def run_ccs_scan():
    """
    Main function to run the complete CCS scan.
    """
    print("\n" + "=" * 80)
    print("STARTING CARNIVAL CORE SCORE SCAN")
    print("=" * 80)
    
    # Step 1: Fetch all market data
    market_data = fetch_all_data(TICKERS)
    
    if len(market_data) == 0:
        print("ERROR: No market data fetched. Check your internet connection.")
        return None
    
    # Step 2: Process all tickers (indicators + scores)
    results = process_all_tickers(market_data)
    
    if len(results) == 0:
        print("ERROR: No results generated. Check processing logic.")
        return None
    
    # Step 3: Generate formatted report
    generate_report(results, TICKERS)
    
    return results

# Execute the scan
print("All modules loaded successfully!")
print("\nTo run the daily scan, execute: run_ccs_scan()")
print("\nYou can also modify TICKERS dictionary above to add/remove instruments.")

In [ ]:
# ============================================================================
# RUN THE SCAN
# ============================================================================

# Run the scan
results = run_ccs_scan()